# ML Pipeline Preparation
Here I create a machine learning pipeline.
### 1. Import libraries and load data from database.
The steps I will folow will be: 
- Import the necessary Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [89]:
# import libraries
import numpy as np
import pandas as pd
import re
import sqlalchemy 

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem import *
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('stopwords')
nltk.download('wordnet')

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
import pickle

[nltk_data] Downloading package words to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/michaelreinhardme.com/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [84]:
print(f'The nltk version is {nltk.__version__}.')
# print('The scikit-learn version is {}.'.format(sklearn.__version__))
print(f'The sklearn version is {sklearn.__version__}')
# print(f'The version of pickle is {pickle.__version__}')

The nltk version is 3.4.4.
The sklearn version is 0.21.2


In [3]:
#I load the data from the database
#sqlite:///
engine = sqlalchemy.create_engine('sqlite:///data/DisasterResponse.db')

#read it into a Pandas DataFrame
df = pd.read_sql_table('disaster', engine) 

In [4]:
#inspect
df.tail()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
26172,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26173,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26174,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26175,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
26176,30265,A radical shift in thinking came about as a re...,None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.shape

(26177, 40)

So there are 26,177 observations and 40 variables, just like in the last notebook. Which is good. Now we get the value counts of the qualitative variables:

In [6]:
for i in ['genre','related','request']:
    print(f'the value counts for "{i}" are:\n{df[i].value_counts()}\n')

the value counts for "genre" are:
news      13036
direct    10747
social     2394
Name: genre, dtype: int64

the value counts for "related" are:
1    19873
0     6117
2      187
Name: related, dtype: int64

the value counts for "request" are:
0    21714
1     4463
Name: request, dtype: int64



#### Split the data into independent and dependent variables

In [7]:
X = df.loc[:,'message']
y = df.iloc[:,5:]

In [8]:
X.shape

(26177,)

In [9]:
y.shape

(26177, 35)

In [10]:
type(X)

pandas.core.series.Series

In [11]:
list(y.columns.values)

['request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

### 2. Write a tokenization function to process your text data

In [90]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
pstemmer = nltk.PorterStemmer()

In [91]:
import re
url_regex = 'http[s]?://(?:[a-zA-Z][0-9]|[$-_@.&+]|[!*\(\),]|[?:%[0-9a-fA-F][0-9a-fA-F])'


In [92]:
def tokenize(text):
    #remove urs and replace with placeholder
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'url_placeholder')
    
    #normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #tokenize text
    tokens = word_tokenize(text)
    
    #lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    #stem the works
    tokens = [pstemmer.stem(word) for word in tokens]
    

    return tokens

In [86]:
def normalize(text):
    #remove urs and replace with placeholder
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'url_placeholder')
    
    #normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #tokenize text
#     tokens = word_tokenize(text)
    
    #lemmatize andremove stop words
    text = [lemmatizer.lemmatize(word) for word in text if word not in stop_words]
    
    

    return text

In [87]:
# X_normalized = X.apply(lambda x: normalize(x))

In [93]:
X_tokenized = X.apply(lambda x: tokenize(x))

In [49]:
print(X_tokenized.head())
print(type(X_tokenized))
print(X_tokenized.index)
print(X_tokenized.values[:2])
print(X_tokenized.shape)

0    [weather, update, cold, front, cuba, could, pa...
1                                          [hurricane]
2                             [looking, someone, name]
3    [un, report, leogane, 80, 90, destroyed, hospi...
4    [say, west, side, haiti, rest, country, today,...
Name: message, dtype: object
<class 'pandas.core.series.Series'>
RangeIndex(start=0, stop=26177, step=1)
[list(['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'])
 list(['hurricane'])]
(26177,)


So we see that `X_tokenized` is a list pandas Series of lists, each list containing the words in a tweet. 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [17]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list[0]:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
            
            
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [ ]:
CountVectorizer()

In [94]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [95]:
# pipeline1 = Pipeline([
#     ('features', FeatureUnion([

#         ('text_pipeline', Pipeline([
#             ('vect', CountVectorizer(tokenizer=tokenize)),
#             ('tfidf', TfidfTransformer())
#         ])),

#         ('starting_verb', StartingVerbExtractor())
#     ])),

#     ('clf', MultiOutputClassifier(RandomForestClassifier()))
    
#     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [58]:
for i in X:
    print(i)
    break

Weather update - a cold front from Cuba that could pass over Haiti


In [72]:
for i, j in enumerate(X_normalized):
    X_normalized[i] = ' '.join(j)



X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, random_state=42)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

AttributeError: 'Series' object has no attribute 'join'

In [41]:
X_train.shape
print(type(X_train))
print(X_train.head())

<class 'pandas.core.series.Series'>
4365     he SMS 3-health - and cleaned 4-reflection and...
8474     I am in the street, I have a house all my card...
11621    Hello.. late good morning. Its a beautiful sun...
17804    A trickle of coins exchanging hands, perhaps a...
3458     There are some people that I know in central P...
Name: message, dtype: object


In [42]:
# X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_tokenized, y, random_state=42)

In [43]:
X_train.shape
print(type(X_train))
print(X_train.head())

<class 'pandas.core.series.Series'>
4365     [sm, 3, health, cleaned, 4, reflection, analys...
8474     [street, house, card, lost, must, make, able, ...
11621    [hello, late, good, morning, beautiful, sunny,...
17804    [trickle, coin, exchanging, hand, perhaps, cru...
3458     [people, know, central, port, au, prince, bloc...
Name: message, dtype: object


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [73]:
pipeline.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: T

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

In [74]:
y_pred = pipeline.predict(X_test)

In [75]:
accuracy_score(y_test, y_pred)

0.33384262796027503

### 6. Improve your model
Use grid search to find better parameters. 

In [25]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__

In [45]:
parameters = {
        
        'clf__estimator__max_depth': [15, 25],
        'clf__estimator__n_estimators': [100, 150] 
    }

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [27]:
cv.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [28]:
y_pred = cv.predict(X_test)

In [29]:
cv.best_params_

{'clf__estimator__max_depth': 15, 'clf__estimator__n_estimators': 75}

In [30]:
for i, label in enumerate(y_test.columns):
    print(label)
    print(confusion_matrix(y_pred[:,i] ,y_test.values[:,i]))

request
[[5455 1089]
 [   0    1]]
offer
[[6516   29]
 [   0    0]]
aid_related
[[3887 2151]
 [  51  456]]
medical_help
[[6063  482]
 [   0    0]]
medical_products
[[6233  312]
 [   0    0]]
search_and_rescue
[[6384  161]
 [   0    0]]
security
[[6439  106]
 [   0    0]]
military
[[6343  202]
 [   0    0]]
child_alone
[[6545]]
water
[[6155  390]
 [   0    0]]
food
[[5865  679]
 [   0    1]]
shelter
[[5980  565]
 [   0    0]]
clothing
[[6449   96]
 [   0    0]]
money
[[6398  147]
 [   0    0]]
missing_people
[[6474   71]
 [   0    0]]
refugees
[[6336  209]
 [   0    0]]
death
[[6249  295]
 [   0    1]]
other_aid
[[5708  837]
 [   0    0]]
infrastructure_related
[[6142  403]
 [   0    0]]
transport
[[6247  298]
 [   0    0]]
buildings
[[6235  310]
 [   0    0]]
electricity
[[6401  144]
 [   0    0]]
tools
[[6505   40]
 [   0    0]]
hospitals
[[6482   63]
 [   0    0]]
shops
[[6510   35]
 [   0    0]]
aid_centers
[[6470   75]
 [   0    0]]
other_infrastructure
[[6281  264]
 [   0    0]]
w

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [66]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [67]:
parameters = {
        'tfidf__norm': ['l2'],
        'tfidf__sublinear_tf': [False],
        'tfidf__smooth_idf': [False],
        'clf__estimator__n_estimators': [20],
        'clf__estimator__criterion' :['gini'],
        'clf__estimator__max_depth' : [1],
        'clf__estimator__max_features': ['log2']
    }

cv = GridSearchCV(pipeline, parameters)

Now fit the model

In [68]:
cv.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [69]:
y_pred = cv.predict(X_test)
cv.best_params_

{'clf__estimator__criterion': 'gini',
 'clf__estimator__max_depth': 1,
 'clf__estimator__max_features': 'log2',
 'clf__estimator__n_estimators': 20,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': False,
 'tfidf__sublinear_tf': False}

In [102]:
accuracy_score(y_test, y_pred)

0.4423223834988541

In [103]:
for i, label in enumerate(y_test.columns):
    print(label)
    print(confusion_matrix(y_test.values[:,i], y_pred[:,i] ))

request
[[5417    0]
 [1128    0]]
offer
[[6514    0]
 [  31    0]]
aid_related
[[3852    0]
 [2693    0]]
medical_help
[[6037    0]
 [ 508    0]]
medical_products
[[6233    0]
 [ 312    0]]
search_and_rescue
[[6364    0]
 [ 181    0]]
security
[[6444    0]
 [ 101    0]]
military
[[6335    0]
 [ 210    0]]
child_alone
[[6545]]
water
[[6133    0]
 [ 412    0]]
food
[[5816    0]
 [ 729    0]]
shelter
[[5958    0]
 [ 587    0]]
clothing
[[6457    0]
 [  88    0]]
money
[[6413    0]
 [ 132    0]]
missing_people
[[6467    0]
 [  78    0]]
refugees
[[6323    0]
 [ 222    0]]
death
[[6257    0]
 [ 288    0]]
other_aid
[[5692    0]
 [ 853    0]]
infrastructure_related
[[6119    0]
 [ 426    0]]
transport
[[6235    0]
 [ 310    0]]
buildings
[[6230    0]
 [ 315    0]]
electricity
[[6413    0]
 [ 132    0]]
tools
[[6506    0]
 [  39    0]]
hospitals
[[6481    0]
 [  64    0]]
shops
[[6516    0]
 [  29    0]]
aid_centers
[[6459    0]
 [  86    0]]
other_infrastructure
[[6258    0]
 [ 287    0]]
w

In [74]:
import xgboost as xgb

In [85]:
clf = xgb.XGBClassifier()

In [94]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(xgb.XGBClassifier()))
])

In [95]:
parameters = {
        'tfidf__norm': ['l1','l2'],
        'tfidf__sublinear_tf': [True, False],
        'tfidf__smooth_idf': [True, False]
    }

cv = GridSearchCV(pipeline, parameters)

In [96]:
cv.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

In [97]:
pred = cv.predict(X_test)

In [98]:
cv.best_params_

{'tfidf__norm': 'l2', 'tfidf__smooth_idf': True, 'tfidf__sublinear_tf': False}

In [99]:
accuracy_score(y_test, y_pred)

0.4423223834988541

In [101]:
for i, label in enumerate(y_test.columns):
    print(label)
    print(confusion_matrix(y_test.values[:,i], y_pred[:,i] ))

request
[[5417    0]
 [1128    0]]
offer
[[6514    0]
 [  31    0]]
aid_related
[[3852    0]
 [2693    0]]
medical_help
[[6037    0]
 [ 508    0]]
medical_products
[[6233    0]
 [ 312    0]]
search_and_rescue
[[6364    0]
 [ 181    0]]
security
[[6444    0]
 [ 101    0]]
military
[[6335    0]
 [ 210    0]]
child_alone
[[6545]]
water
[[6133    0]
 [ 412    0]]
food
[[5816    0]
 [ 729    0]]
shelter
[[5958    0]
 [ 587    0]]
clothing
[[6457    0]
 [  88    0]]
money
[[6413    0]
 [ 132    0]]
missing_people
[[6467    0]
 [  78    0]]
refugees
[[6323    0]
 [ 222    0]]
death
[[6257    0]
 [ 288    0]]
other_aid
[[5692    0]
 [ 853    0]]
infrastructure_related
[[6119    0]
 [ 426    0]]
transport
[[6235    0]
 [ 310    0]]
buildings
[[6230    0]
 [ 315    0]]
electricity
[[6413    0]
 [ 132    0]]
tools
[[6506    0]
 [  39    0]]
hospitals
[[6481    0]
 [  64    0]]
shops
[[6516    0]
 [  29    0]]
aid_centers
[[6459    0]
 [  86    0]]
other_infrastructure
[[6258    0]
 [ 287    0]]
w

### 9. Export your model as a pickle file

In [49]:
disaster_model = 'models/finalized_model.sav'
pickle.dump(cv, open(disaster_model, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.